In [1]:
import fitz
import re
import pandas as pd
import json
import io
import os
import csv

In [2]:
def writeToCsv(csvfile, data):
    if os.path.isfile(csvfile):
        with open(csvfile, "a", newline='', encoding='utf-8') as csvr:
            writer = csv.writer(csvr, delimiter = "*", quotechar = '~', quoting = csv.QUOTE_NONNUMERIC, lineterminator=">")
            writer.writerows(data)
    else:
        with open(csvfile, "w", newline='', encoding='utf-8') as csvr:
            writer = csv.writer(csvr, delimiter = "*", quotechar = '~', quoting = csv.QUOTE_NONNUMERIC, lineterminator='>')
            writer.writerows(data)

In [8]:
def pdf_text_retrieval(pdf_path):
    recording = False
    grab_date = False
    content = ""
    process = "(italic|oblique)"
    titles = "bold"
    dates = "^([0-9]|[0-9][0-9])\s(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)\s[0-9][0-9][0-9][0-9]($|\s$)"
    page_no = "^[0-9]+(\.\s$|\s$|$)"
    time = "^\d\d:\d\d(\s$|$)"
    blockfonts = []
    blocksizes = []
    speech = ""
    speaker = ""
    speakerhold = ""
    terminate = "^\s*(Meeting closed at|Members who would like a printed copy of the|This is the final edition of the Official Report of this meeting)"
    scoParl = "(^\s+|^)Scottish Parliament($|\s+$)"
    nameEnd = ":(\s+$|$)"
    sentEnd = "(\.|!|\?)(\s+$|$)"
    officialTitlesNames = "(The\s*Presiding\s*Officer(\s*\(|\().*\):|The\s*Deputy\s*Presiding\s*Officer(\s*\(|\().*\):|The\s*Convener(\s*\(|\().*\):|The\s*Deputy\s*Convener(\s*\(|\().*\):)|\(The\s*Convener\)(\s:|:)"
    officialTitlesOnly = "(The\s*Presiding\s*Officer(\s:|:)|The\s*Deputy\s*Presiding\s*Officer(\s:|:)|The\s*Convener(\s:|:)|The\s*Deputy\s*Convener(\s:|:))"
    prev_title = False
    prev_italics = False
    prev_normal = False
    gathered_text = []
    
    try:
        doc = fitz.open(pdf_path, filetype='pdf')
    
        for i in range(0, len(doc)):
            page = doc.load_page(i)
            text_instances = page.get_text("dict")["blocks"]
            try:
                for block in text_instances:
                    title = ""
                    content = ""
                    cont_title = False
                    cont_italics = False
                    cont_normal = False
                    try:
                        for line in block["lines"]:
                            fonts = []
                            sizes = []
                            try:
                                for span in line["spans"]:
                                    text = span["text"]
                                    size = span["size"]
                                    font = span["font"].lower()

                                    if (re.match(dates, text) == None) and ((re.match(page_no, text) == None) or size < 9)  and (re.match(time, text) == None):
                                        if bool(re.search(titles, font)):
                                            title = " ".join((title, text))
                                            cont_title = True

                                        else:
                                            content = " ".join((content, text))
                                            if bool(re.search(process, font)):
                                                cont_italics = True
                                            elif bool(re.search(titles, font)) != True:
                                                cont_normal = True

                                        sizes.append(size)
                                        fonts.append(font)
                            except:
                                with open("pdfProcErrorLog.txt", 'a') as f:
                                    f.write("Error processing SPAN on page " + str(i) + " in doc: " + pdf_path +"\n")

                    except:
                        with open("pdfProcErrorLog.txt", 'a') as f:
                            f.write("Error processing LINE on page " + str(i) + " in doc: " + pdf_path +"\n")



                    if recording:
    #                     print("________________________\n","Contains Title: ", cont_title, prev_title, "\nContains Italics: ", cont_italics, prev_italics, "\nContains Normal: ", cont_normal, prev_normal, "\n---------\n",title, "\n", content, "\n______________________")
                #         for i in fonts:
                #             blockfonts.append(i)
                #         for i in sizes:
                #             blocksizes.append(i)

                #         blockfonts = set(blockfonts)
                #         blocksizes = set(blocksizes)

                        if (cont_title == True) and (cont_normal == True):

                            if (len(speech) > 1) and (len(speaker)>1):
                                if bool(re.search(officialTitlesNames, speaker)):
    #                                 print("Found name:" , " ", speaker)
                                    mc = speaker
                                elif bool(re.search(officialTitlesOnly, speaker)):
    #                                 print("\nCont Title, Cont Normal, OTO\n", speaker, "\n")
                                    speaker = mc

                                gathered_text.append([speaker, speech, session_date])
    #                             print(speaker," \n", speech, "\n", session_date, "\n", "------------------")

    #                         print(speaker)
                            if len(speakerhold) > 0:
    #                            print("_________\n", "SPEAKERHOLD", "\n", "___________\n")
                                speaker = " ".join((speakerhold, title))
                                speakerhold = ""
                            else:
                                speaker = title
                            speech = content
                            new_page = False

                        elif (cont_title == True) and (cont_italics == False) and (cont_normal == False):

                            if bool(re.search(nameEnd, title)):
    #                             print("______________\n", "ONE ROW TITLE\n", "_____________\n")
                                if (len(speech) > 1) and (len(speaker) > 1):
                                    if bool(re.search(officialTitlesNames, speaker)):
                                        mc = speaker
                                    elif bool(re.search(officialTitlesOnly, speaker)):
                                        speaker = mc
                                    gathered_text.append([speaker, speech, session_date])
    #                                 print(speaker," \n", speech, "\n", session_date, "\n", "------------------")

                                speaker = " ".join((speakerhold, title))
                                speech = ""
                                speakerhold = ""
                            else:
                                speakerhold = title
                            new_page = False

                        elif (cont_title == True) and (cont_italics == True) and (cont_normal == False):
                            content = ""
                            new_page = False


                        elif (cont_title == False) and (cont_italics == True) and (cont_normal == False):
                                if grab_date:
                                    session_date = content
                                    grab_date = False
    #                                 print("The Date is ", session_date, "!\n")
                                elif bool(re.search(terminate, content)):
                #                     print(blockfonts, " ", blocksizes, "\n", speaker, "\n", speech, "\n", "- - - - - - - - - - - - - - - - - - - \n")

                                    if bool(re.search(officialTitlesOnly, speaker)):
                                        speaker = mc
                                    gathered_text.append([speaker, speech, session_date]) 
    #                                 print(speaker," \n", speech, "\n", session_date, "\n", "------------------")


                                    content = ""

                                    doc.close()
    #                                 try:
    #                                     writeToCsv(csv, gathered_text)
    #                                 except:
    #                                     print("CSV issues")
    #                                 print("\n\n --------TERMINATE----------\n\n")
    #                                 print(gathered_text)
                                    return gathered_text

                                else:
                                    content = ""
                                new_page = False


                        elif (cont_title == False) and (cont_italics == True) and (cont_normal == True):
    #                         print("---------------ITALICS/NORMAL-----------\n", content, "------------------------\n")
    #                         print("Grab_date: ", grab_date)
                            if grab_date:
                                session_date = content
                                grab_date = False
    #                             print("The Date is ", session_date, "!\n")
                            elif bool(re.search(terminate, content)):
                                if bool(re.search(officialTitlesNames, speaker)):
    #                                 print("Title/Name\n", content)
                                    mc = speaker
                                elif bool(re.search(officialTitlesOnly, speaker)):
    #                                 print("Title/NoName\n", content)
                                    speaker = mc
                                gathered_text.append([speaker, speech, session_date])
    #                             print(speaker," \n", speech, "\n", session_date, "\n", "------------------")



                                content = ""

                                doc.close()

    #                             print("\n\n --------TERMINATE----------\n\n")
                                return gathered_text

                            else:
                                content = ""
                                new_page = False


                        elif (prev_title == False) and (prev_normal == False) and (prev_italics == True) and (cont_title == False):
                            if bool(re.search(terminate, content)):
                                if bool(re.search(officialTitlesNames, speaker)):
                                    mc = speaker
                                elif bool(re.search(officialTitlesOnly, speaker)):
                                    speaker = mc
                                gathered_text.append([speaker, speech, session_date])
    #                             print(speaker," \n", speech, "\n", session_date, "\n", "------------------")

    #                             print("\n\n --------TERMINATE----------\n\n")

                                doc.close()
                                return gathered_text

                            else:
                                content = ""
                                cont_title = prev_title
                                cont_italics = prev_italics
                                cont_normal = prev_normal
                                new_page = False


                        elif (cont_title == False) and (cont_italics == False) and (cont_normal==False):
                            content = ""
                            new_page = True
                            cont_title = prev_title
                            cont_italics = prev_italics
                            cont_normal = prev_normal

                        elif (cont_title == False) and (cont_normal == True):
    #                         print("-----ALL NORMAL----\n", content, "\n----------------")                   
                            if bool(re.search(terminate, content)):
                                if bool(re.search(officialTitlesNames, speaker)):
                                    mc = speaker
                                elif bool(re.search(officialTitlesOnly, speaker)):
                                    speaker = mc
                                gathered_text.append([speaker, speech, session_date])
    #                             print(speaker," \n", speech, "\n", session_date, "\n", "------------------")

    #                             print("\n\n --------TERMINATE----------\n\n")

                                doc.close()
                                return gathered_text
                            elif bool(re.search(sentEnd, speech)):
                                speech = "\n".join((speech, content))

                            else:
                                speech = " ".join((speech, content))
                            new_page = False


                        prev_title = cont_title
                        prev_italics = cont_italics
                        prev_normal = cont_normal
                    else:
    #                     print("\nUnrecorded\n")
                        if bool(re.match(scoParl, title)):
                            recording = True
                            grab_date = True
            except:
                with open("pdfProcErrorLog.txt", 'a') as f:
                    f.write("Error processing BLOCK on page " + str(i) + " in doc: " + pdf_path + "\n")
    
    except:
        with open("unopenablePDFs.txt", 'a') as f:
            f.write("Unable to open pdf " + pdf_path + "\n")
    
    try:
        doc.close()
    except:
        pass

In [4]:
def pdf_churn(folder, csv):
    
    cwd = os.getcwd()
    wd = os.path.join(cwd, folder)
    #We can iterate through all the files in the directory...
    for file in os.listdir(wd):
        pdf = os.path.join(wd, file)
        data = pdf_text_retrieval(pdf)
        try:
            writeToCsv(csv, data)
        except:
            with open("pdf_churn_errors.txt", "a") as errorLog:
                if type(data) == type(None):
                    error = "Document " + pdf + " returned Nonetype data.\n"
                    errorLog.write(error)
                else:
                    error = "problem with " + pdf + "\n"
                    errorLog.write(error)

In [9]:
pdf_churn("OR_Storage/", "Scots_Parl.csv")

In [133]:
data = pdf_text_retrieval("1999 - Scot_Parl.pdf")
new_data = data_to_dict(data)
with io.open('test.json', 'w', encoding='utf-8') as output:
    json.dump(data, output, ensure_ascii=False)

The Date is   Wednesday 12 May 1999 !



 --------TERMINATE----------




In [131]:
data = pdf_text_retrieval("2023 - Scot_Parl.pdf")
new_data = data_to_dict(data)
with io.open('test.json', mode = 'r+', encoding='utf-8') as output:
    output.seek(0, 2)
    position = output.tell() -1
    output.seek(position)
    output.write(" ,{}]".format(json.dumps(data)))

The Date is   Thursday 11 May 2023   !



 --------TERMINATE----------




In [147]:
data = pdf_text_retrieval("OR_Storage/OR-141.pdf")
# writeToCsv("trial.csv", data)

The Date is   Tuesday 27 February 2007  !



In [58]:
print(data)

None


In [47]:
names = ["Speaker", "Speech", "Date"]
holder = pd.read_csv("trial.csv", delimiter="*", names=names, lineterminator='>')
holder.head(-5)

,Speaker,Speech,Date
0,~ General Question Time The Presiding Office...,~ Good morning. Our first item of business is...,~ Thursday 11 May 2023 ~
1,~ Gender Pay Gap David Torrance (Kirkcaldy) ...,~ To ask the Scottish Government what steps ...,~ Thursday 11 May 2023 ~
2,~ The Cabinet Secretary for Wellbeing Ec...,~ Figures from the Office for National Statist...,~ Thursday 11 May 2023 ~
3,~ David Torrance: ~,~ Has the Scottish Government made a recent ...,~ Thursday 11 May 2023 ~
4,~ Neil Gray: ~,~ I thank David Torrance for raising that im...,~ Thursday 11 May 2023 ~
...,...,...,...
276,~ Deputy Presiding Officers The Presiding Off...,"~ This time, members should remember to inser...",~ Wednesday 12 May 1999~
277,~ The Presiding Officer:~,~ The election will proceed in accordance wit...,~ Wednesday 12 May 1999~
278,~ The Presiding Officer:~,~ The voting time for this round is now close...,~ Wednesday 12 May 1999~
279,~ The Presiding Officer:~,~ In the first round of voting in the electio...,~ Wednesday 12 May 1999~


In [46]:
data = pdf_text_retrieval("1999 - Scot_Parl.pdf")
# writeToCsv("trial.csv", data)

The Date is   Wednesday 12 May 1999 !



 --------TERMINATE----------




In [68]:
gieit = "(The Presiding Officer(\s:|:)|The Deputy Presiding Officer(\s:|:)|The Convener(\s:|:)|The Deputy Convener(\s:|:))"
test = "David Torrance (Kirkcaldy) (SNP):"
fuckit = "(The Presiding Officer\(\w*\)|The Deputy Presiding Officer \(\w*\)|The Convener\(\w*\)|The Deputy Convener\(\w*\))"
bool(re.search(test, fuckit))

False

In [35]:
data[-1]

[' The Presiding Officer: ',
 ' The final question is,   that motion S6M-08683, in the name of Shona  Robison, on a financial resolution for the Charities  (Regulation and Administration) (Scotland) Bill, be  agreed to. ',
 ' Thursday 11 May 2023  ']

In [104]:
columns = ["Speaker", "Speech", "date"]
df = pd.DataFrame(data)
df.head()

,0-0,0-9,0-17,0-28,0-38,0-47,0-55,0-65,0-78,0-85,...,0-225,0-227,0-228,0-229,0-233,0-237,0-243,0-245,0-247,0-250
member,"Christine Grahame (Midlothian South, Tweedd...",Maurice Golden:,Màiri McAllan:,Kevin Stewart:,The First Minister:,The First Minister:,The First Minister:,The First Minister (Humza Yousaf):,Fiona Hyslop:,The First Minister:,...,Foysol Choudhury (Lothian) (Lab):,John Mason (Glasgow Shettleston) (SNP):,The Deputy Presiding Officer:,Pam Duncan-Glancy:,Fergus Ewing (Inverness and Nairn) (SNP):,The Deputy Presiding Officer:,The Deputy Presiding Officer:,The Presiding Officer (Alison Johnstone):,Financial Resolution The Presiding Officer ...,The Presiding Officer:
speech,To ask the Scottish Government what measures...,"I thank the minister for that answer, but i...",The answer is a resounding no. \n The Scottis...,"I want to see the issue progressed, but the...","Once again, the Scottish Government is havi...",It is quite incredible that when we give a ...,I will say what the SNP is interested in. W...,We will give that consideration. Littering ...,"As the First Minister has acknowledged, the...",If it comes back to the question of the for...,...,"Before I begin, I would like to declare an ...",I appreciate the opportunity to take part i...,"I call Maggie Chapman, who joins us remotel...",I will.,"You are so kind, Presiding Officer. Thank yo...","Thank you, Mr Ewing, for doing justice to m...","Thank you, Mr O’Kane. I am not sure that yo...",I call Shirley-Anne Somerville to wind up.,The next item of business is consideration o...,"The final question is, that motion S6M-0868..."
date,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,...,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023,Thursday 11 May 2023


In [114]:
with io.open('test.json', 'a', encoding='utf-8') as output:
    json.dump(data, output, ensure_ascii=False)

In [132]:
trial = pd.read_json("test.json", encoding='utf-8')
trial.head()

,0,1,2,3,4,5,6,7,8,9,...,243,244,245,246,247,248,249,250,251,252
0,The Temporary Clerk (Paul Grice):,"Welcome to this, the first meeting of the Sco...",Wednesday 12 May 1999,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Oaths and Affirmations Dr Winnie Ewing (Olde...,Oath taking by members will take place from 9...,Wednesday 12 May 1999,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Mr Alex Salmond (Banff and Buchan) (SNP):,The Scottish National party parliamentary gro...,Wednesday 12 May 1999,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Dennis Canavan (Falkirk West):,I want to make it clear that I believe in the...,Wednesday 12 May 1999,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Robin Harper (Lothians) (Green):,The Green party would have preferred to add t...,Wednesday 12 May 1999,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [53]:
regex = "(The Presiding Officer(\s*\(|\().*\):|The Deputy Presiding Officer(\s*\(|\().*\):|The Convener(\s*\(|\().*\):|The Deputy Convener(\s*\(|\().*\):)"
reg2 = "The Presiding Officer(\s*\(|\().*\):"
source = "Preliminary Stage   The Convener(Phil Gallie) : "
bool(re.search(regex, source))

False

In [112]:
doc = fitz.open("C:\\Users\\admin.BDL014\\Documents\\Scottish Parliament Data\\OR_Storage\\OR-144.pdf", filetype='pdf')
page = doc.load_page(4)
text_instances = page.get_text("dict")["blocks"]
# for block in text_instances:
#     print(block, "\n")
sentence = ""
for line in text_instances[8]["lines"]:
    for span in line["spans"]:
        print(span)
        sentence = sentence + span["text"]
    
# print(sentence)


terminate = "(^\s+|^)(Meeting closed at|Members who would like a printed copy of the Official Report to be forwarded to them should give notice|This is the final edition of the Official Report of this meeting)"

bool(re.search(terminate,"  Members who would like a printed copy of the Official Report to be forwarded to them should give notice at the"))

{'size': 10.020000457763672, 'flags': 16, 'font': 'Arial,Bold', 'color': 0, 'ascender': 1.0700000524520874, 'descender': -0.3070000112056732, 'text': 'The ', 'origin': (65.22000122070312, 234.0799560546875), 'bbox': (65.22000122070312, 223.35855102539062, 85.82112884521484, 237.15609741210938)}
{'size': 10.020000457763672, 'flags': 16, 'font': 'Arial,Bold', 'color': 0, 'ascender': 1.0700000524520874, 'descender': -0.3070000112056732, 'text': 'Convener ', 'origin': (94.96939086914062, 234.0799560546875), 'bbox': (94.96939086914062, 223.35855102539062, 143.87701416015625, 237.15609741210938)}
{'size': 10.020000457763672, 'flags': 16, 'font': 'Arial,Bold', 'color': 0, 'ascender': 1.0700000524520874, 'descender': -0.3070000112056732, 'text': '(Charlie ', 'origin': (153.03529357910156, 234.0799560546875), 'bbox': (153.03529357910156, 223.35855102539062, 192.99502563476562, 237.15609741210938)}
{'size': 10.020000457763672, 'flags': 16, 'font': 'Arial,Bold', 'color': 0, 'ascender': 1.07000005

True

In [20]:
test = 'Members who would like a printed copy of the Official Report to be forwarded to them should give notice'
test2 = "^This is the final edition of the Official Report of this meeting"
phrase = "Members who would like a printed copy of the Official Report to be forwarded to them should give notice at the"
phrase2 = "This is the final edition of the Official Report of this meeting. It is part of the Scottish Parliament Official Report archive and has been sent for legal deposit."
bool(re.search(test2, phrase2))

True